# Market dominance

Did Amazon exert market dominance by creating and promoting their own products? 

Can we show that creating/promoting Amazon brands for products harmed competition by reducing the diversity of sellers from whom consumers purchased the product?

e.g. compare purchases for Amazon branded "soy sauce" to other sellers of "soy sauce".

Note this example is not in our data.

#### High level analysis plan
- Find Amazon branded products in our purchases dataset that 
    - have threshold amount of total purchases and months of purchases
    - where purchases start after a few quarters
- And where there are at least a threshold number of other sellers selling in period beforehand
- In each [month|quarter|period] measure the number of different sellers for product type
- Track change in number of sellers before vs after Amazon product entered market

#### Outcome
Found our dataset does not support this study.

After attempting to create a dataset of Amazon products and similar products, found that the only candidate Amazon products to study were introduced before or during the beginning of 2018.
This means there is not a point of comparison before the introduction of these products.


In [189]:
from datetime import datetime
import math

from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Amazon purchases data

### Preprocessing:
- Restrict data to 2018-01 to 2022-12
- Label purchases data by quarter & month
    - month label: starts at 2018-01: 1, then 2018-02: 2, etc.
    - quarter label: maps months to quarters
        - month 1-3: 1, 4-6: 2, 7-9: 3, 10-12: 4, 13-15: 5, etc
- Remove gift cards
- Label products as Amazon products: Amazon product: boolean
    - Search by keywords in title
        - "Amazon brand", "Amazon  basics", "Solimo", "Presto!"

In [2]:
amzn_data_fpath = '../data/amazon-data/amazon-data-cleaned.csv'
amzn_data = pd.read_csv(amzn_data_fpath, index_col=[0])
# add year to data
amzn_data['year'] = pd.to_datetime(amzn_data['Order Date']).apply(lambda d: d.year)
# label purchases by month starting from 2018-01
# map yyyy-mm to month int
month_labels = {ym: i for (i, ym) in enumerate(sorted(amzn_data['yyyy-mm'].unique()), start=1)}
amzn_data['month'] = amzn_data['yyyy-mm'].map(month_labels)
# quarters: map months to quarters
# month 1-3: 1, 4-6: 2, 7-9: 3, 10-12: 4, 13-15: 5, etc
amzn_data['quarter'] = amzn_data['month'].apply(lambda m: math.ceil(m/3))
# restrict the data to our columns of interest and reorder
amzn_data = amzn_data[[
    'Order Date', 'yyyy-mm', 'year', 'month', 'quarter',
    'Title','ASIN/ISBN (Product Code)', 'Category', 'unit price', 'Quantity', 
    'Survey ResponseID',
]]
print('%s total purchases from N=%s customers' % (len(amzn_data), amzn_data['Survey ResponseID'].nunique()))
# limit the data to range: 2018-01 to 2022-12
print('restricting to purchases made 2018-01 to 2022-12')
amzn_data = amzn_data[(amzn_data['year'] >= 2018) & (amzn_data['year'] <= 2022)]
print('%s total purchases from N=%s customers' % (len(amzn_data), amzn_data['Survey ResponseID'].nunique()))
# peek at it:
amzn_data.drop(['Survey ResponseID'], axis=1).head(3)

1850717 total purchases from N=5027 customers
restricting to purchases made 2018-01 to 2022-12
1810793 total purchases from N=5026 customers


,Order Date,yyyy-mm,year,month,quarter,Title,ASIN/ISBN (Product Code),Category,unit price,Quantity
0,2018-02-21,2018-02,2018,2,1,Suburban World: The Norling Photos,0873516095,ABIS_BOOK,7.93,1.0
1,2018-02-21,2018-02,2018,2,1,NaN,B004S7EZR0,NaN,3.53,1.0
2,2018-03-05,2018-03,2018,3,1,1952 Back In The Day - 24-page Greeting Card /...,193938012X,ABIS_BOOK,5.99,1.0


Remove gift cards

exclude by 
- category GIFT_CARD
- title indicating gift card/code: amazon reload, gift code, etc

(Not all gift cards/codes have category GIFT_CARD)

In [3]:
def is_gift_code(row):
    cat = str(row['Category'])
    if cat in ['GIFT_CARD']:
        return True
    title = str(row['Title']).lower()
    if ('gift code' in title) or ('gift card' in title) or ('digital code' in title):
        return True
    if ('amazon reload' in title):
        return True
    return False

gift_card_purchases = amzn_data[amzn_data.apply(is_gift_code, axis=1)]
print('Dropping gift card/code purchases')
amzn_data = amzn_data[~amzn_data.apply(is_gift_code, axis=1)]
print('%s total purchases from N=%s customers' % (len(amzn_data), amzn_data['Survey ResponseID'].nunique()))

print('Most popular gift card/code purchases by total purchases with prices:')
gift_card_purchases.groupby(
    ['ASIN/ISBN (Product Code)', 'Title', 'Category']
)['unit price'].agg(['min', 'median', 'count']).sort_values('count', ascending=False).head()

Dropping gift card/code purchases
1768650 total purchases from N=5025 customers
Most popular gift card/code purchases by total purchases with prices:


,,,min,median,count
ASIN/ISBN (Product Code),Title,Category,,,
B00IX1I3G6,Amazon.com Gift Card Balance Reload,GIFT_CARD,0.5,5.00,11336
B086KKT3RX,Amazon Reload,ABIS_GIFT_CARD,0.5,5.11,5098
B07PCMWTSG,Amazon.com eGift Card,GIFT_CARD,1.0,20.00,1390
B004LLIKVU,Amazon.com eGift Card,GIFT_CARD,1.0,15.00,1388
B086KKT3RX,Amazon Reload,GIFT_CARD,0.5,4.00,921


Label products as Amazon products

See https://en.wikipedia.org/wiki/List_of_Amazon_brands

- Amazon product: boolean
- Search by keywords in title
    - "Amazon brand", "Amazon basics", "Solimo", "Presto!"

In [26]:
# https://en.wikipedia.org/wiki/List_of_Amazon_brands
amazon_brand_keywords = [
    'amazon brand', 'amazon elements', 'amazon essentials', 'amazon basics', 
    'solimo', 'presto!',
    # These products tend to have Amazon brand in the title so may not be necessary to search for
    # Don't include Amazon Fresh for now because this is fresh food delivery
    'pinzon', 'mama bear', 'happy belly', 
    'goodthreads', '206 collective', 'core 10', 'mae', # apparel specific
    '365 ',
]

def is_amazon_brand(title, keywords=amazon_brand_keywords):
    t = str(title).lower()
    return any([kw in t for kw in keywords])

amzn_data['Amazon brand'] = amzn_data['Title'].apply(is_amazon_brand)
amazon_brand_purchases = amzn_data[amzn_data['Amazon brand']]
print('%s total Amazon product purchases from N=%s customers' % 
      (len(amazon_brand_purchases), amazon_brand_purchases['Survey ResponseID'].nunique()))
# non_amazon_brand_purchases = 

56066 total Amazon product purchases from N=3861 customers


In [27]:
non_amazon_brand_purchases = amzn_data[~amzn_data['Amazon brand']]
print('%s total non Amazon product purchases from N=%s customers' % 
      (len(non_amazon_brand_purchases), non_amazon_brand_purchases['Survey ResponseID'].nunique()))

1712584 total non Amazon product purchases from N=5025 customers


Inspect top Amazon branded products

In [28]:
print('Most popular Amazon brand products by total purchases with prices:')
display(amazon_brand_purchases.groupby(
    ['ASIN/ISBN (Product Code)', 'Title', 'Category']
)['unit price'].agg(['min', 'median', 'max', 'count']).sort_values('count', ascending=False).head())

print('Most common categories by total purchases with prices:')
display(amazon_brand_purchases.groupby(
    ['Category']
)['unit price'].agg(['min', 'median', 'max', 'count']).sort_values('count', ascending=False).head(10))

Most popular Amazon brand products by total purchases with prices:


,,,min,median,max,count
ASIN/ISBN (Product Code),Title,Category,,,,
B00MNV8E0C,"Amazon Basics 48 Pack AA High-Performance Alkaline Batteries, 10-Year Shelf Life, Easy to Open Value Pack",BATTERY,6.52,14.99,16.93,561
B00LH3DMUO,"Amazon Basics 36 Pack AAA High-Performance Alkaline Batteries, 10-Year Shelf Life, Easy to Open Value Pack",BATTERY,7.74,10.99,18.09,557
B074CRK54X,"Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Mega Rolls = 112 regular rolls",TOILET_PAPER,15.55,22.25,26.76,310
B00NTCH52W,"Amazon Basics 20 Pack AA High-Performance Alkaline Batteries, 10-Year Shelf Life, Easy to Open Value Pack",BATTERY,7.32,9.03,14.24,262
B074CRK54X,"Amazon Brand - Presto! Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Family Mega Rolls = 120 regular rolls",TOILET_PAPER,17.80,22.25,26.76,260


Most common categories by total purchases with prices:


,min,median,max,count
Category,,,,
BATTERY,1.51,10.99,64.56,3476
ELECTRONIC_CABLE,1.23,8.09,89.91,2700
VEGETABLE,0.49,2.29,18.68,2648
DAIRY_BASED_CHEESE,0.59,3.19,9.79,2401
SHIRT,4.27,16.00,229.99,1754
TOILET_PAPER,3.69,22.25,53.07,1279
GROCERY,0.75,2.69,23.21,1157
FOOD,0.79,2.99,13.75,1067
PANTS,4.36,22.90,66.09,1034


#### Make the dataset of products to study

Restrict to Amazon products st
- at least M=12 unique months of purchases
- First purchase date after month min_M (for min_M=12: first purchase after 2019-01; month > 12)
- at least 3 unique purchasers

For each Amazon product:
- Record  first purchase date
- Record first review date
- Check: Do they match up??
    - problem: cannot find first review date
- Gather keywords to look up similar products


First:
Collect products by first month, total unique months, purchases

Observation:
- Many amazon brand products already being purchsed by 2018-01

In [31]:
amazon_brand_purchases.groupby(
    ['ASIN/ISBN (Product Code)', 'Title']
)['yyyy-mm'].agg(['min','nunique', 'max','count']).merge(
    amazon_brand_purchases.groupby(
        ['ASIN/ISBN (Product Code)']
    )['Survey ResponseID'].nunique().rename('nunique').to_frame(),
    left_index=True, right_index=True, suffixes=('_months', '_purchasers')
).sort_values('count', ascending=False).head()

,,min,nunique_months,max,count,nunique_purchasers
ASIN/ISBN (Product Code),Title,,,,,
B00MNV8E0C,"Amazon Basics 48 Pack AA High-Performance Alkaline Batteries, 10-Year Shelf Life, Easy to Open Value Pack",2018-01,60,2022-12,561,301
B00LH3DMUO,"Amazon Basics 36 Pack AAA High-Performance Alkaline Batteries, 10-Year Shelf Life, Easy to Open Value Pack",2018-01,60,2022-12,557,363
B074CRK54X,"Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Mega Rolls = 112 regular rolls",2018-05,53,2022-11,310,130
B00NTCH52W,"Amazon Basics 20 Pack AA High-Performance Alkaline Batteries, 10-Year Shelf Life, Easy to Open Value Pack",2018-01,57,2022-12,262,175
B074CRK54X,"Amazon Brand - Presto! Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Family Mega Rolls = 120 regular rolls",2018-01,58,2022-12,260,130


Collect the products that meet criteria

Note that sometimes the same product code had different titles (evolved) so group by product code without title.

In [55]:
# min_first_m = 10 # start after month 9
# min_unique_m = 12 # have at least 12 months of purchases
min_first_q = 4 # start after quarter 3
min_unique_q = 3 # have at least 3 quarters of purcahses data
min_purchases = 50 
min_unique_purchasers = 3

abp_months = amazon_brand_purchases.groupby(
    ['ASIN/ISBN (Product Code)']
)['quarter'].agg(['min','nunique', 'max','count']).merge(
    amazon_brand_purchases.groupby(
        ['ASIN/ISBN (Product Code)']
    )['Survey ResponseID'].nunique().rename('nunique').to_frame(),
    left_index=True, right_index=True, suffixes=('_quarters', '_purchasers')
)
abp_months = abp_months[
    (abp_months['min'] >= min_first_q) & (abp_months['nunique_quarters'] >= min_unique_q)
]
print('%s Amazon products meet purchase periods criteria with a total of %s transactions' % 
      (len(abp_months), abp_months['count'].sum()))

abp_months = abp_months[(abp_months['count'] >= min_purchases)]
print('%s Amazon products also meet criteria for total purchases count with a total of %s transactions' % 
      (len(abp_months), abp_months['count'].sum()))

abp_months = abp_months[(abp_months['nunique_purchasers'] >= min_unique_purchasers)]
print('%s Amazon products also meet criteria for unique purchasers with a total of %s transactions' % 
      (len(abp_months), abp_months['count'].sum()))

abp_months.sort_values('count', ascending=False).head()

2640 Amazon products meet purchase periods criteria with a total of 29800 transactions
63 Amazon products also meet criteria for total purchases count with a total of 4482 transactions
63 Amazon products also meet criteria for unique purchasers with a total of 4482 transactions


,min,nunique_quarters,max,count,nunique_purchasers
ASIN/ISBN (Product Code),,,,,
B07FGBSF45,4,14,17,206,74
B077L5RXJY,6,14,20,152,61
B0787K2XWZ,4,17,20,144,132
B07F246YG1,5,15,20,133,62
B075Y8ZV89,6,15,20,129,60


Collect the items and their info

Make table with unique product codes with: 
title, code, category, 
- keep first month of purchase
- add: unique months, total purchases, min price, max price, median price


In [106]:
amazon_brand_products_study = amzn_data[
    amzn_data['ASIN/ISBN (Product Code)'].isin(abp_months.index)
].sort_values('month').drop_duplicates(
    ['ASIN/ISBN (Product Code)'], keep='first'
).set_index('ASIN/ISBN (Product Code)')[[
    'Title','Category', 'yyyy-mm', 'quarter','month',
]].rename(columns={'yyyy-mm': 'first yyyy-mm', 'quarter': 'first quarter', 'month': 'first month'})
amazon_brand_products_study['unique quarters'] = amazon_brand_products_study.index.map(abp_months['nunique_quarters'])
amazon_brand_products_study['unique purchasers'] = amazon_brand_products_study.index.map(abp_months['nunique_purchasers'])
amazon_brand_products_study['purchases'] = amazon_brand_products_study.index.map(abp_months['count'])

amazon_brand_products_study = amazon_brand_products_study.merge(
    amzn_data[amzn_data['ASIN/ISBN (Product Code)'].isin(abp_months.index)].groupby(
        'ASIN/ISBN (Product Code)'
    )['unit price'].agg(['min','max','median']).rename(
        columns={'min': 'price_min', 'max': 'price_max', 'median': 'price_median'}
    ), left_index=True, right_index=True)
pd.set_option('display.max_colwidth', 80)
amazon_brand_products_study.head(3)

,Title,Category,first yyyy-mm,first quarter,first month,unique quarters,unique purchasers,purchases,price_min,price_max,price_median
ASIN/ISBN (Product Code),,,,,,,,,,,
B0787K2XWZ,Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery,BATTERY,2018-10,4,10,17,132,144,3.26,7.54,5.49
B07FGBSF45,"Amazon Brand - Solimo 2-Ply Toilet Paper, 6 Count (Pack of 5)",TOILET_PAPER,2018-10,4,10,14,74,206,12.99,22.71,19.99
B07DC4RZPY,Amazon Basics MFi-Certified Lightning to USB A Cable for Apple iPhone and iP...,ELECTRONIC_CABLE,2018-10,4,10,9,40,53,3.89,9.49,6.49


Should exclude fresh food items / groceries ordered via Amazon Fresh/Whole foods. Exclude these by category and exclude "365" brand.

In [107]:
print('Dropping "365" brand items from study')
amazon_brand_products_study = amazon_brand_products_study[~amazon_brand_products_study['Title'].apply(lambda t: '365' in t)]
print('%s Amazon products with a total of %s transactions' % 
      (len(amazon_brand_products_study), amazon_brand_products_study['purchases'].sum()))

Dropping "365" brand items from study
37 Amazon products with a total of 2895 transactions


In [108]:
# Exclude perishable groceries
exclude_cats = [
    'DAIRY_BASED_BUTTER', 'DAIRY_BASED_CHEESE', 'DAIRY_BASED_CREAM', 'DAIRY_BASED_DRINK', 
    'DAIRY_BASED_YOGURT',
    'EGG', 'LEGUME', 'VEGETABLE',
    'GROCERY',
]
print('Dropping Amazon products in grocery / fresh food categories:\n', exclude_cats)
amazon_brand_products_study = amazon_brand_products_study[~amazon_brand_products_study['Category'].isin(exclude_cats)]
print('\n%s Amazon products with a total of %s transactions' % 
      (len(amazon_brand_products_study), amazon_brand_products_study['purchases'].sum()))

Dropping Amazon products in grocery / fresh food categories:
 ['DAIRY_BASED_BUTTER', 'DAIRY_BASED_CHEESE', 'DAIRY_BASED_CREAM', 'DAIRY_BASED_DRINK', 'DAIRY_BASED_YOGURT', 'EGG', 'LEGUME', 'VEGETABLE', 'GROCERY']

16 Amazon products with a total of 1337 transactions


In [109]:
# Collect product code, title, category
print('Some of these may be the same-ish product. Sorted by Category and Title to better identify.')
pd.set_option('display.max_colwidth', None)
amazon_brand_products_study.sort_values(['Category', 'Title'])#[['Title','Category', 'unique quarters', 'quarter','purchases']]

Some of these may be the same-ish product. Sorted by Category and Title to better identify.


,Title,Category,first yyyy-mm,first quarter,first month,unique quarters,unique purchasers,purchases,price_min,price_max,price_median
ASIN/ISBN (Product Code),,,,,,,,,,,
B07NJQ9RRG,"Amazon Brand - Solimo Epsom Salt Soaking Aid, Lavender Scented, 3 Pound",BATHWATER_ADDITIVE,2019-05,6,17,13,32,50,4.27,5.51,4.390
B0787K2XWZ,Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery,BATTERY,2018-10,4,10,17,132,144,3.26,7.54,5.490
B079V7F2BP,"Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Breakfast Blend, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,2018-10,4,10,17,16,94,14.85,39.48,29.100
B07WSTZ95G,"Amazon Brand - Happy Belly Original Saltine Crackers, 16 Ounce",CRACKER,2020-06,10,30,10,34,58,0.99,3.50,2.075
B07PC8CC3B,"Amazon Brand - Happy Belly Vegetable Oil, 48 Fl Oz",EDIBLE_OIL_VEGETABLE,2020-01,9,25,12,38,55,1.84,5.79,3.670
B07DC4RZPY,Amazon Basics MFi-Certified Lightning to USB A Cable for Apple iPhone and iPad - 3 Feet (0.9 Meters) - White,ELECTRONIC_CABLE,2018-10,4,10,9,40,53,3.89,9.49,6.490
B07CZCQ481,"Amazon Basics 5-Blade MotionSphere Razor for Men with Dual Lubrication and Precision Trimmer, Handle & 16 Cartridges (Cartridges fit Amazon Basics Razor Handles only) (Previously Solimo)",MANUAL_SHAVING_RAZOR,2019-09,7,21,14,42,50,12.76,24.70,17.990
B07D123YBX,"Amazon Brand - Solimo Probiotic 5 Billion CFU, 8 strains with 60 mg Prebiotic Blend, 60 Vegetarian Capsules, 2 Month Supply, Supports Healthy Digestion",NUTRITIONAL_SUPPLEMENT,2019-05,6,17,13,19,50,7.02,19.98,9.960
B09BWFX1L6,"Amazon Basics 2-Ply Paper Towels, Flex-Sheets, 6 Rolls (Pack of 2), 12 Value Rolls total (Previously Solimo)",PAPER_TOWEL,2022-03,17,51,4,48,55,13.68,20.49,16.220


Note that the above products are all of distinct types of items.
- Toilet paper:
    - B095CN96JS: Amazon Basics 2-Ply Toilet Paper 5 Packs, 6 Rolls per pack (30 Rolls total) (Previously Solimo)
    - B07FGBSF45: Amazon Brand - Solimo 2-Ply Toilet Paper, 6 Count (Pack of 5)	TOILET_PAPER

For other things that are similar (e.g. 2 types of waste bags) it is fine to just rely on the presence of the first -- this is when Amazon first entered the market for this item.

For each Amazon product:
- collect keywords to look up similar products
    - test
- map amazon product code to list of product codes for similar non-amazon products that match keywords


TODO / questions:
- should pay attention to size or cost?




In [112]:
# Printing the following to drop into a spreadsheet
amazon_brand_products_study = amazon_brand_products_study.sort_values(['Category', 'Title'])
amazon_brand_products_study[['Title','Category','first quarter']]

,Title,Category,first quarter
ASIN/ISBN (Product Code),,,
B07NJQ9RRG,"Amazon Brand - Solimo Epsom Salt Soaking Aid, Lavender Scented, 3 Pound",BATHWATER_ADDITIVE,6
B0787K2XWZ,Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery,BATTERY,4
B079V7F2BP,"Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Breakfast Blend, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,4
B07WSTZ95G,"Amazon Brand - Happy Belly Original Saltine Crackers, 16 Ounce",CRACKER,10
B07PC8CC3B,"Amazon Brand - Happy Belly Vegetable Oil, 48 Fl Oz",EDIBLE_OIL_VEGETABLE,9
B07DC4RZPY,Amazon Basics MFi-Certified Lightning to USB A Cable for Apple iPhone and iPad - 3 Feet (0.9 Meters) - White,ELECTRONIC_CABLE,4
B07CZCQ481,"Amazon Basics 5-Blade MotionSphere Razor for Men with Dual Lubrication and Precision Trimmer, Handle & 16 Cartridges (Cartridges fit Amazon Basics Razor Handles only) (Previously Solimo)",MANUAL_SHAVING_RAZOR,7
B07D123YBX,"Amazon Brand - Solimo Probiotic 5 Billion CFU, 8 strains with 60 mg Prebiotic Blend, 60 Vegetarian Capsules, 2 Month Supply, Supports Healthy Digestion",NUTRITIONAL_SUPPLEMENT,6
B09BWFX1L6,"Amazon Basics 2-Ply Paper Towels, Flex-Sheets, 6 Rolls (Pack of 2), 12 Value Rolls total (Previously Solimo)",PAPER_TOWEL,17


For each item map product code to:
- first quarter in purchases data
- duplicate versions to exclude
- keyterms
- title

For each product, first search within amazon brand products for other products 
- check if purchased earlier than first quarter for given product
    - if so --> cannot include product

In [148]:
# drop the duplicate items and just keep the first version
amazon_brand_products_study['duplicates'] = [[]]*len(amazon_brand_products_study)
# record duplicate 2-ply toilet paper
amazon_brand_products_study.at['B07FGBSF45','duplicates'] = ['B095CN96JS']
# record duplicate waste bags
amazon_brand_products_study.at['B07BJ4CYR4','duplicates'] = ['B093X8QQ12']
# drop duplicates to just keep the fist version
amazon_brand_products_study = amazon_brand_products_study.drop(['B095CN96JS', 'B093X8QQ12'])
amazon_brand_products_study = amazon_brand_products_study.sort_values(['Category', 'Title'])
print('%s Amazon brand products after removing duplicates' % len(amazon_brand_products_study))
amazon_brand_products_study[['Title','Category','first quarter', 'duplicates']]

14 Amazon brand products after removing duplicates


,Title,Category,first quarter,duplicates
ASIN/ISBN (Product Code),,,,
B07NJQ9RRG,"Amazon Brand - Solimo Epsom Salt Soaking Aid, Lavender Scented, 3 Pound",BATHWATER_ADDITIVE,6,[]
B0787K2XWZ,Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery,BATTERY,4,[]
B079V7F2BP,"Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Breakfast Blend, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,4,[]
B07WSTZ95G,"Amazon Brand - Happy Belly Original Saltine Crackers, 16 Ounce",CRACKER,10,[]
B07PC8CC3B,"Amazon Brand - Happy Belly Vegetable Oil, 48 Fl Oz",EDIBLE_OIL_VEGETABLE,9,[]
B07DC4RZPY,Amazon Basics MFi-Certified Lightning to USB A Cable for Apple iPhone and iPad - 3 Feet (0.9 Meters) - White,ELECTRONIC_CABLE,4,[]
B07CZCQ481,"Amazon Basics 5-Blade MotionSphere Razor for Men with Dual Lubrication and Precision Trimmer, Handle & 16 Cartridges (Cartridges fit Amazon Basics Razor Handles only) (Previously Solimo)",MANUAL_SHAVING_RAZOR,7,[]
B07D123YBX,"Amazon Brand - Solimo Probiotic 5 Billion CFU, 8 strains with 60 mg Prebiotic Blend, 60 Vegetarian Capsules, 2 Month Supply, Supports Healthy Digestion",NUTRITIONAL_SUPPLEMENT,6,[]
B09BWFX1L6,"Amazon Basics 2-Ply Paper Towels, Flex-Sheets, 6 Rolls (Pack of 2), 12 Value Rolls total (Previously Solimo)",PAPER_TOWEL,17,[]


In [171]:
# From a the spreadsheet I set up
# https://docs.google.com/spreadsheets/d/13dXD_SZCu4rTk0wQJjj0y-wiDNvdGgdJ71znEsRSTfk/edit#gid=304600266

# ASIN keywords
asin_keyterms = {
    "B07NJQ9RRG": [["epsom"], ["salt"], ["soak"], [" scent"]],
    "B0787K2XWZ": [["3v", "3 v", "3.0 volt", "3 volt"], ["lithium"], ["coin"], ["battery", "batteries"]],
    "B079V7F2BP": [["coffee"], ["pod"], ["k-cup", "k cup"], ["keurig"]],
    "B07WSTZ95G": [["saltine cracker"]],
    "B07PC8CC3B": [["vegetable oil"]],
    "B07DC4RZPY": [["mfi certified", "mfi-certified"], ["cable", "cord"], ["lightning"], ["usb"], ["apple"]],
    "B07CZCQ481": [["razor"], [" men"]],
    "B07D123YBX": [["probiotic"], ["prebiotic"], ["vegetarian"]],
    "B09BWFX1L6": [["paper towel"]],
    "B07H53W5WP": [["baby wipe"], ["unscented"]],
    "B07F246YG1": [["disinfect", "sanitize"],["wipe"],[" scent"]],
    "B095CN96JS": [["toilet paper"]],
    "B07FGBSF45": [["toilet paper"]],
    "B093X8QQ12": [["drawstring"], ["trash bag"]],
    "B07BJ4CYR4": [["drawstring"], ["trash bag"]],
    "B07VD4KN28": [["distilled water"]],
}
amazon_brand_products_study['keyterms'] = amazon_brand_products_study.index.map(asin_keyterms)
amazon_brand_products_study[['Title', 'keyterms','Category','first quarter', 'duplicates']]

,Title,keyterms,Category,first quarter,duplicates
ASIN/ISBN (Product Code),,,,,
B07NJQ9RRG,"Amazon Brand - Solimo Epsom Salt Soaking Aid, Lavender Scented, 3 Pound","[[epsom], [salt], [soak], [ scent]]",BATHWATER_ADDITIVE,6,[]
B0787K2XWZ,Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery,"[[3v, 3 v, 3.0 volt, 3 volt], [lithium], [coin], [battery, batteries]]",BATTERY,4,[]
B079V7F2BP,"Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Breakfast Blend, Compatible with Keurig 2.0 K-Cup Brewers","[[coffee], [pod], [k-cup, k cup], [keurig]]",COFFEE,4,[]
B07WSTZ95G,"Amazon Brand - Happy Belly Original Saltine Crackers, 16 Ounce",[[saltine cracker]],CRACKER,10,[]
B07PC8CC3B,"Amazon Brand - Happy Belly Vegetable Oil, 48 Fl Oz",[[vegetable oil]],EDIBLE_OIL_VEGETABLE,9,[]
B07DC4RZPY,Amazon Basics MFi-Certified Lightning to USB A Cable for Apple iPhone and iPad - 3 Feet (0.9 Meters) - White,"[[mfi certified, mfi-certified], [cable, cord], [lightning], [usb], [apple]]",ELECTRONIC_CABLE,4,[]
B07CZCQ481,"Amazon Basics 5-Blade MotionSphere Razor for Men with Dual Lubrication and Precision Trimmer, Handle & 16 Cartridges (Cartridges fit Amazon Basics Razor Handles only) (Previously Solimo)","[[razor], [ men]]",MANUAL_SHAVING_RAZOR,7,[]
B07D123YBX,"Amazon Brand - Solimo Probiotic 5 Billion CFU, 8 strains with 60 mg Prebiotic Blend, 60 Vegetarian Capsules, 2 Month Supply, Supports Healthy Digestion","[[probiotic], [prebiotic], [vegetarian]]",NUTRITIONAL_SUPPLEMENT,6,[]
B09BWFX1L6,"Amazon Basics 2-Ply Paper Towels, Flex-Sheets, 6 Rolls (Pack of 2), 12 Value Rolls total (Previously Solimo)",[[paper towel]],PAPER_TOWEL,17,[]


In [172]:
# for example
B0787K2XWZ_battery_keyterms = amazon_brand_products_study.loc['B0787K2XWZ']['keyterms']
B0787K2XWZ_battery_keyterms

[['3v', '3 v', '3.0 volt', '3 volt'],
 ['lithium'],
 ['coin'],
 ['battery', 'batteries']]

In [173]:
"""
For each amazon product we have a set of key terms that must be in the title of similar products
For each key term there might be multiple versions
e.g. ["3 volt", "3.0 volt", "3v"] can all be used to find a 3 volt battery product
key terms are thus passed as a list of lists where for each list item in the outter list, 
at least one element in the list item must be in the title.

e.g. to find a product similar to Amazon's "3 Volt Lithium Coin Cell Battery"
[["3v", "3 v", "3.0 volt", "3 volt"], ["lithium"], ["coin"], ["battery", "batteries"]]
"""

def is_similar_product(title, keyterms=None):
    t = str(title).lower()
    return all([any([k in t for k in kterm]) for kterm in keyterms])

def find_similar_products(keyterms, purchases_df, verbose=False):
    p = purchases_df[purchases_df['Title'].apply(is_similar_product, keyterms=keyterms)]
    if verbose:
        print('%s purchases for %s distinct products over %s distinct months' % (
            len(p), p['ASIN/ISBN (Product Code)'].nunique(), p['yyyy-mm'].nunique()
        ))
    return p
    

For each Amazon brand product, check for other Amazon brand products with same key terms that were actually sold earlier.

If found, disqualifies the product from study.

In [182]:
duplicated_amzn_brand_pcodes = []
non_duplicated_amazon_brand_pcodes = []

for pcode in amazon_brand_products_study.index:
    title = amazon_brand_products_study.loc[pcode]['Title']
    keyterms = amazon_brand_products_study.loc[pcode]['keyterms']
    first_q = amazon_brand_products_study.loc[pcode]['first quarter']
    print('%s: "%s"; first quarter: %s\nsearching with keyterms:\n%s' % (
        pcode, title, first_q, keyterms
    ))
    ps = find_similar_products(keyterms, amazon_brand_purchases)
    if ps['quarter'].min() < first_q:
        print('Found Amazon branded products sold earlier!')
        display(ps.groupby(
            ['ASIN/ISBN (Product Code)', 'Title', 'Category']
        )['quarter'].agg(['min', 'max', 'count']).sort_values('min'))
        duplicated_amzn_brand_pcodes += [pcode]
        continue
    non_duplicated_amazon_brand_pcodes += [pcode]

B07NJQ9RRG: "Amazon Brand - Solimo Epsom Salt Soaking Aid, Lavender Scented, 3 Pound"; first quarter: 6
searching with keyterms:
[['epsom'], ['salt'], ['soak'], [' scent']]
B0787K2XWZ: "Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery"; first quarter: 4
searching with keyterms:
[['3v', '3 v', '3.0 volt', '3 volt'], ['lithium'], ['coin'], ['battery', 'batteries']]
B079V7F2BP: "Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Breakfast Blend, Compatible with Keurig 2.0 K-Cup Brewers"; first quarter: 4
searching with keyterms:
[['coffee'], ['pod'], ['k-cup', 'k cup'], ['keurig']]
Found Amazon branded products sold earlier!


,,,min,max,count
ASIN/ISBN (Product Code),Title,Category,,,
B079V7QCX4,"Amazon Brand - 100 Ct. Solimo Medium Roast Coffee Pods, Colombian, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,2,20,21
B079V8CKDM,"Amazon Brand - 100 Ct. Solimo Dark Roast Coffee Pods, Compatible with Keurig 2.0 K-Cup Brewers 100 Count(Pack of 1)",COFFEE,2,20,107
B079V9JDQB,"Amazon Brand - 100 Ct. Solimo Dark Roast Coffee Pods, French Roast, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,2,20,93
B079VC7NLP,"Amazon Brand - 100 Ct. Solimo Medium Roast Coffee Pods, Kona Blend, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,2,19,16
B079V9JCQ2,"Amazon Brand - 100 Ct. Solimo Medium-Dark Roast Coffee Pods, House Blend, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,3,11,9
B079V7358L,"Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Morning Light, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,4,13,5
B079V7F2BP,"Amazon Brand - 100 Ct. Solimo Light Roast Coffee Pods, Breakfast Blend, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,4,20,94
B07NQFC1G9,"Amazon Brand - 100 Ct. Happy Belly Dark Roast Coffee Pods, French Roast, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,7,20,12
B07K1XSTTD,"Amazon Brand - 24 Ct. Solimo Coffee Pods, Colombian, Compatible with Keurig 2.0 K-Cup Brewers",COFFEE,7,19,3


B07WSTZ95G: "Amazon Brand - Happy Belly Original Saltine Crackers, 16 Ounce"; first quarter: 10
searching with keyterms:
[['saltine cracker']]
Found Amazon branded products sold earlier!


,,,min,max,count
ASIN/ISBN (Product Code),Title,Category,,,
B074MG5VVJ,"365 Everyday Value, Saltine Crackers, 16 Ounce",CRACKER,2,2,1
B07CJTNG6H,"Amazon Brand - Solimo Saltine Crackers, 16 ounce",GROCERY,5,5,2
B07WSTZ95G,"Amazon Brand - Happy Belly Original Saltine Crackers, 16 Ounce",CRACKER,10,20,58
B08BD3Z8T7,"Amazon Brand - Happy Belly Wheat Saltine Crackers, 16 Ounce",CRACKER,13,20,3


B07PC8CC3B: "Amazon Brand - Happy Belly Vegetable Oil, 48 Fl Oz"; first quarter: 9
searching with keyterms:
[['vegetable oil']]
B07DC4RZPY: "Amazon Basics MFi-Certified Lightning to USB A Cable for Apple iPhone and iPad - 3 Feet (0.9 Meters) - White"; first quarter: 4
searching with keyterms:
[['mfi certified', 'mfi-certified'], ['cable', 'cord'], ['lightning'], ['usb'], ['apple']]
Found Amazon branded products sold earlier!


...

B07F246YG1: "Amazon Basics Disinfecting Wipes, Lemon Scent & Fresh Scent, Sanitizes/Cleans/Disinfects/Deodorizes, 85 Count: Pack of 3 (Previously Solimo)"; first quarter: 5
searching with keyterms:
[['disinfect', 'sanitize'], ['wipe'], [' scent']]
B07FGBSF45: "Amazon Brand - Solimo 2-Ply Toilet Paper, 6 Count (Pack of 5)"; first quarter: 4
searching with keyterms:
[['toilet paper']]
Found Amazon branded products sold earlier!


min   
ASIN/ISBN (Product Code) Title                                                                                                                                                         Category                    
B074CRK54X               Amazon Brand - Presto! 313-Sheet Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Family Mega Rolls = 120 regular rolls                            TOILET_PAPER            1  \
                         Amazon Brand - Presto! Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Family Mega Rolls = 120 regular rolls                                      TOILET_PAPER            1   
B074CT964K               Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Strong, 6 Count                                                                                TOILET_PAPER            1   
B074CTMZ3Y               Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Soft, 6 Count                                                                                  TOILET_PAPER            1   
B074CR89PR               Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Strong, 6 Count (Pack of 4), 24 Count = 96 Regular Rolls                                       TOILET_PAPER            2   
                         Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Strong, 6 count (pack of 4), 24 Count=96 regular rolls                                         TOILET_PAPER            2   
B074CRK54X               Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 4), 24 Mega Rolls = 112 regular rolls                                   TOILET_PAPER            2   
B074CT964Q               Amazon Brand - Presto! 154-Sheet Roll Toilet Paper, Ultra-Strong, 12 Count (For Small Roll Holders)                                                           TOILET_PAPER            2   
B074CTF6X6               Amazon Brand - Presto! 154-Sheet Roll Toilet Paper, Ultra-Soft, 12 Count (For Small Roll Holders)                                                             HEALTH_PERSONAL_CARE    2   
B07BSTWW6M               Amazon Brand - Presto! 176-Sheet Roll Toilet Paper, Ultra-Soft, 12 Count (For Small Roll Holders)                                                             TOILET_PAPER            3   
B07BSRCMFN               Amazon Brand - Presto! 176-Sheet Roll Toilet Paper, Ultra-Soft, 48 Count (For Small Roll Holders),12 Count (Pack of 4)                                        TOILET_PAPER            3   
B07BSSVFMY               Amazon Brand - Presto! 176-Sheet Roll Toilet Paper, Ultra-Strong, 12 Count (For Small Roll Holders)                                                           TOILET_PAPER            3   
B073Q66Y7J               Amazon Basics Free Standing Bathroom Toilet Paper Holder Stand with Reserve, Silver Nickel                                                                    TOILET_PAPER_HOLDER     4   
B07FGBSF45               Amazon Brand - Solimo 2-Ply Toilet Paper, 6 Count (Pack of 5)                                                                                                 TOILET_PAPER            4   
B07DVBRNPK               Amazon Brand - Solimo 2-Ply Toilet Paper, 350 Sheets per Roll, 6 Count                                                                                        TOILET_PAPER            4   
B07FGBSF45               Amazon Brand - Solimo 2-Ply Toilet Paper, 6 Count (Pack of 5)                                                                                                 HEALTH_PERSONAL_CARE    7   
B07QV93RLQ               Amazon Brand - Presto! 308-Sheet Mega Roll Toilet Paper, Ultra-Strong, 6 Count                                                                                TOILET_PAPER            8   
B07QV942J6               Amazon Brand - Presto! 313-Sheet Mega Roll Toilet Paper, Ultra-Soft, 6 Count (Pack of 1), 6 Family Mega Rolls = 30 Regular Rolls (Packaging may vary)   

B07BJ4CYR4: "Amazon Brand - Solimo Multipurpose Drawstring Trash Bags, 30 Gallon, 50 Count"; first quarter: 4
searching with keyterms:
[['drawstring'], ['trash bag']]
Found Amazon branded products sold earlier!


,,,min,max,count
ASIN/ISBN (Product Code),Title,Category,,,
B07BJ49P7B,"Solimo Trash Bags (Unscented, Tall Kitchen Drawstring, 13 Gallon, 200 Count)",WASTE_BAG,3,20,75
B07BJ4L4QL,"Solimo Trash Bags (Unscented, Tall Kitchen Drawstring, 13 Gallon, 120 Count)",WASTE_BAG,3,17,83
B07BJ4D1W1,"Amazon Brand - Solimo Tall Kitchen Drawstring Trash Bags, 13 Gallon, 90 Count",WASTE_BAG,3,13,32
B07BJ4639P,"Amazon Brand - Solimo Tall Kitchen Drawstring Trash Bags, Clean Fresh Scent, 13 Gallon, 80 Count",WASTE_BAG,4,17,37
B07BJ4LQXR,"Amazon Brand - Solimo Tall Kitchen Drawstring Trash Bags, 13 Gallon, 45 Count",WASTE_BAG,4,17,13
B07BJ4CYR4,"Amazon Brand - Solimo Multipurpose Drawstring Trash Bags, 30 Gallon, 50 Count",WASTE_BAG,4,15,112
B07BJ4KFW9,"Amazon Brand - Solimo Tall Kitchen Drawstring Trash Bags, Clean Fresh Scent, 13 Gallon, 200 Count",WASTE_BAG,5,17,7
B07BJ4KFWG,"Amazon Brand - Solimo Lawn & Leaf Drawstring Trash Bags, 39 Gallon, 40 Count",WASTE_BAG,5,15,9
B07N84HM1W,"Amazon Brand - Solimo Trash Bags (Unscented, Tall Kitchen Drawstring, 13 Gallon, 45 Count)",WASTE_BAG,5,17,30


B07VD4KN28: "Amazon Brand - Happy Belly Distilled Water, 1 Gallon (128 oz)"; first quarter: 10
searching with keyterms:
[['distilled water']]


In [186]:
print('%s non duplicated Amazon brand product codes' % len(non_duplicated_amazon_brand_pcodes))
non_duplicated_amazon_brand_pcodes

6 non duplicated Amazon brand product codes


['B07NJQ9RRG',
 'B0787K2XWZ',
 'B07PC8CC3B',
 'B07D123YBX',
 'B07F246YG1',
 'B07VD4KN28']

In [188]:
amazon_brand_products_study.loc[non_duplicated_amazon_brand_pcodes]#[['Title', 'keyterms','Category','first quarter', 'duplicates']]

,Title,Category,first yyyy-mm,first quarter,first month,unique quarters,unique purchasers,purchases,price_min,price_max,price_median,duplicates,keyterms
ASIN/ISBN (Product Code),,,,,,,,,,,,,
B07NJQ9RRG,"Amazon Brand - Solimo Epsom Salt Soaking Aid, Lavender Scented, 3 Pound",BATHWATER_ADDITIVE,2019-05,6,17,13,32,50,4.27,5.51,4.39,[],"[[epsom], [salt], [soak], [ scent]]"
B0787K2XWZ,Amazon Basics 4 Pack CR2032 3 Volt Lithium Coin Cell Battery,BATTERY,2018-10,4,10,17,132,144,3.26,7.54,5.49,[],"[[3v, 3 v, 3.0 volt, 3 volt], [lithium], [coin], [battery, batteries]]"
B07PC8CC3B,"Amazon Brand - Happy Belly Vegetable Oil, 48 Fl Oz",EDIBLE_OIL_VEGETABLE,2020-01,9,25,12,38,55,1.84,5.79,3.67,[],[[vegetable oil]]
B07D123YBX,"Amazon Brand - Solimo Probiotic 5 Billion CFU, 8 strains with 60 mg Prebiotic Blend, 60 Vegetarian Capsules, 2 Month Supply, Supports Healthy Digestion",NUTRITIONAL_SUPPLEMENT,2019-05,6,17,13,19,50,7.02,19.98,9.96,[],"[[probiotic], [prebiotic], [vegetarian]]"
B07F246YG1,"Amazon Basics Disinfecting Wipes, Lemon Scent & Fresh Scent, Sanitizes/Cleans/Disinfects/Deodorizes, 85 Count: Pack of 3 (Previously Solimo)",SURFACE_CLEANING_WIPE,2019-03,5,15,15,62,133,7.37,12.78,8.99,[],"[[disinfect, sanitize], [wipe], [ scent]]"
B07VD4KN28,"Amazon Brand - Happy Belly Distilled Water, 1 Gallon (128 oz)",WATER,2020-06,10,30,10,21,68,0.79,1.59,0.99,[],[[distilled water]]


Should probably disqualify the disinfecting wipes since they were likely already offered (see "Previously Solimo") and likely wracked up more purchases after COVID.

The only other compelling product is the 3 Volt Lithium Coin Cell Battery.

The other products have few total purchases relative to first quarter ordered and therefore likely flukes.